In [1]:
pip install requests pandas

In [2]:
import requests
import pandas as pd
import time

# === Your Edamam API credentials ===
APP_ID = '59456ca4'
APP_KEY = 'ebeb4beaff17d1a14fcec62a904206a3'

# === Extended Search Terms ===
SEARCH_TERMS = [
    # Core ingredients
    'chicken', 'beef', 'pork', 'salmon', 'shrimp', 'tofu', 'eggs', 'lamb', 'duck', 'turkey',
    'lentils', 'beans', 'peas', 'spinach', 'kale', 'broccoli', 'cauliflower', 'zucchini',
    'tomato', 'potato', 'sweet potato', 'onion', 'carrot', 'bell pepper', 'mushroom', 'garlic',
    'avocado', 'cucumber', 'corn', 'asparagus', 'brussels sprouts', 'eggplant',

    # Fruits and snacks
    'apple', 'banana', 'berries', 'mango', 'grapes', 'peach', 'pineapple', 'orange', 'lemon',
    'fruit salad', 'fruit smoothie', 'energy balls', 'granola bars', 'trail mix',

    # Protein variations
    'ground chicken', 'grilled chicken', 'fried chicken', 'chicken curry', 'chicken stir fry',
    'ground beef', 'beef stew', 'beef chili', 'beef curry', 'beef kebab',
    'grilled salmon', 'baked salmon', 'salmon sushi', 'smoked salmon',
    'tofu curry', 'tofu stir fry', 'tofu scramble', 'tofu salad',

    # Grains and carbs
    'quinoa', 'rice', 'brown rice', 'fried rice', 'basmati rice',
    'pasta', 'spaghetti', 'penne pasta', 'macaroni', 'noodles', 'ramen', 'udon', 'soba',
    'bread', 'whole wheat bread', 'sourdough', 'naan', 'flatbread',

    # Diets
    'vegan', 'vegetarian', 'gluten free', 'dairy free', 'keto', 'paleo', 'low carb', 'high protein',
    'low fat', 'sugar free', 'plant based', 'mediterranean diet', 'clean eating',

    # Cooking methods
    'baked', 'grilled', 'roasted', 'steamed', 'boiled', 'fried', 'sauteed', 'air fryer', 'slow cooker',
    'pressure cooker', 'instant pot',

    # Cuisine types
    'italian', 'mexican', 'indian', 'thai', 'chinese', 'japanese', 'korean', 'vietnamese',
    'french', 'greek', 'spanish', 'lebanese', 'turkish', 'caribbean', 'african', 'american',
    'brazilian', 'russian', 'ethiopian', 'indonesian', 'malaysian', 'moroccan',

    # Dish types
    'soup', 'stew', 'salad', 'sandwich', 'wrap', 'tacos', 'burrito', 'pizza', 'lasagna', 'paella',
    'risotto', 'curry', 'stir fry', 'omelette', 'scrambled eggs', 'pancakes', 'waffles',
    'muffins', 'cookies', 'cake', 'brownies', 'pudding', 'cheesecake', 'ice cream',

    # Meals
    'breakfast', 'brunch', 'lunch', 'dinner', 'snack', 'dessert', 'side dish', 'appetizer', 'main course',

    # Occasion-based
    'christmas dinner', 'thanksgiving turkey', 'easter recipes', 'diwali snacks',
    'halloween treats', 'ramadan recipes', 'eid desserts', 'summer BBQ', 'picnic food',

    # Beverages
    'smoothie', 'protein shake', 'fruit juice', 'detox water', 'iced tea', 'hot chocolate', 'latte', 'coffee drinks',

    # Special types
    'meal prep', 'one pot meals', '5 ingredient recipes', 'budget meals', 'quick recipes', 'low calorie recipes'
]

# Settings
TOTAL_RECIPES_TARGET = 10000
RECIPES_PER_TERM = TOTAL_RECIPES_TARGET // len(SEARCH_TERMS)
RECIPES_PER_REQUEST = 100
BASE_URL = 'https://api.edamam.com/api/recipes/v2'

def fetch_recipes(query, total_recipes):
    all_recipes = []
    next_url = f"{BASE_URL}?type=public&q={query}&app_id={APP_ID}&app_key={APP_KEY}&random=true"

    fetched = 0
    request_count = 0
    requests_needed = (total_recipes + RECIPES_PER_REQUEST - 1) // RECIPES_PER_REQUEST

    while fetched < total_recipes:
        if request_count >= requests_needed:
            break

        response = requests.get(next_url)
        if response.status_code != 200:
            print(f"Error for '{query}': {response.status_code}")
            break

        data = response.json()
        hits = data.get('hits', [])
        if not hits:
            break

        for hit in hits:
            recipe = hit['recipe']
            all_recipes.append({
                'title': recipe.get('label', ''),
                'ingredients': '; '.join(recipe.get('ingredientLines', [])),
                'calories': recipe['totalNutrients'].get('ENERC_KCAL', {}).get('quantity', 0),
                'fat': recipe['totalNutrients'].get('FAT', {}).get('quantity', 0),
                'protein': recipe['totalNutrients'].get('PROCNT', {}).get('quantity', 0),
                'carbs': recipe['totalNutrients'].get('CHOCDF', {}).get('quantity', 0),
                'url': recipe.get('url', ''),
                'servings': recipe.get('yield', 1)
            })
            fetched += 1
            if fetched >= total_recipes:
                break

        next_link = data.get('_links', {}).get('next', {}).get('href')
        if not next_link:
            break
        next_url = next_link

        request_count += 1
        time.sleep(6)

    print(f"Fetched {fetched} recipes for '{query}'")
    return all_recipes

def save_to_csv(recipes, filename):
    df = pd.DataFrame(recipes)
    df.to_csv(filename, index=False)
    print(f"Saved {len(df)} recipes to {filename}")

if __name__ == "__main__":
    for term in SEARCH_TERMS:
        print(f"\n--- Starting '{term}' ---")
        data = fetch_recipes(term, RECIPES_PER_TERM)
        save_to_csv(data, f"{term.replace(' ', '_')}_recipes.csv")

    print("\nAll scraping tasks completed!")



--- Starting 'chicken' ---
Fetched 20 recipes for 'chicken'
Saved 20 recipes to chicken_recipes.csv

--- Starting 'beef' ---
Fetched 20 recipes for 'beef'
Saved 20 recipes to beef_recipes.csv

--- Starting 'pork' ---
Fetched 20 recipes for 'pork'
Saved 20 recipes to pork_recipes.csv

--- Starting 'salmon' ---
Fetched 20 recipes for 'salmon'
Saved 20 recipes to salmon_recipes.csv

--- Starting 'shrimp' ---
Fetched 20 recipes for 'shrimp'
Saved 20 recipes to shrimp_recipes.csv

--- Starting 'tofu' ---
Fetched 20 recipes for 'tofu'
Saved 20 recipes to tofu_recipes.csv

--- Starting 'eggs' ---
Fetched 20 recipes for 'eggs'
Saved 20 recipes to eggs_recipes.csv

--- Starting 'lamb' ---
Fetched 20 recipes for 'lamb'
Saved 20 recipes to lamb_recipes.csv

--- Starting 'duck' ---
Fetched 20 recipes for 'duck'
Saved 20 recipes to duck_recipes.csv

--- Starting 'turkey' ---
Fetched 20 recipes for 'turkey'
Saved 20 recipes to turkey_recipes.csv

--- Starting 'lentils' ---
Fetched 20 recipes for 'l